Importing libraries

In [1]:
from keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance
import h5py
import pandas as pd

Using TensorFlow backend.


Open file

In [ ]:
new_file = h5py.File("../data/mipas_blabeled_reinhold_features/rf_mipas_complete", "r")

Load Brightness Temperature Difference features and labels

In [ ]:
btd_complete_scaled = new_file["btd_complete_scaled"][:]

labels = new_file['labels_bc'][:]

Close file

In [ ]:
new_file.close()

Select training and validation data

In [ ]:
f_test_scaled = btd_complete_scaled[25000:, :]

f_train_scaled = btd_complete_scaled[0:25000, :]

Scale features between in the range [0, 1]

In [ ]:
f_test_scaled = MinMaxScaler().fit_transform(f_test_scaled)

f_train_scaled = MinMaxScaler().fit_transform(f_train_scaled)

Define model

In [ ]:
def nn_model():
    "create a model."
    model = Sequential()
    
    model.add(Dropout(0.1))

    model.add(Dense(output_dim = 500, input_dim=10011, activation='relu'))
    
    model.add(Dropout(0.1))

    model.add(Dense(output_dim = 200, input_dim=500, activation='relu'))

    model.add(Dense(output_dim = 100, input_dim=200, activation='relu'))
        
    model.add(Dense(output_dim = 50, input_dim=100, activation='relu'))
    
    model.add(Dropout(0.1))
    
    model.add(Dense(output_dim = 20, input_dim=50, activation='relu'))

    model.add(Dense(output_dim = 20, input_dim=20, activation='relu'))
    
    model.add(Dropout(0.1))
    
    model.add(Dense(output_dim = 50, input_dim=20, activation='relu'))
    
    model.add(Dense(output_dim = 100, input_dim=50, activation='relu'))

    model.add(Dense(output_dim = 200, input_dim=100, activation='relu'))
    
    model.add(Dropout(0.1))

    model.add(Dense(output_dim = 500, input_dim=200, activation='relu'))
    
    model.add(Dense(output_dim = 10011, input_dim=500, activation='sigmoid'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

Fit model on training data

In [ ]:
my_model = KerasRegressor(build_fn=nn_model, epochs=50)

my_model.fit(f_train_scaled, f_train_scaled) 

Permutations computation

In [ ]:
perm = PermutationImportance(my_model, random_state=1).fit(f_train_scaled,f_train_scaled)

Create pandas dataframe to deal with features columns and set their id to string (needed by eli5 library)

In [ ]:
df = pd.DataFrame(f_train_scaled)

results = [str(i) for i in df.columns]

Show features weights

In [ ]:
eli5.show_weights(perm, feature_names = results)